# **Clasificación de Dígitos Manuscritos - Análisis y Entrenamiento de Modelos Supervisados**

---

## **1. Descripción del problema y del conjunto de datos**

Este proyecto aborda un problema de **clasificación de imágenes**, específicamente la **identificación de dígitos manuscritos** (del 0 al 9) a partir de imágenes en escala de grises.  
El objetivo es construir **modelos de aprendizaje supervisado** capaces de predecir correctamente el dígito que representa cada imagen.

El conjunto de datos proviene de la plataforma **Kaggle** y contiene imágenes de **28x28 píxeles**, cada una representada como una fila con **784 características** (una por píxel), más una columna `label` que indica el dígito correcto.

---

### **Variables:**

- **label:**  
  Entero entre 0 y 9 que representa el dígito manuscrito.

- **pixel0 a pixel783:**  
  Valores entre 0 y 255 que indican la intensidad de cada píxel.

---

Este conjunto de datos es **ampliamente utilizado** para tareas de clasificación de imágenes debido a su simplicidad y relevancia en problemas reales de **visión por computador**.

## **2. Visualización e inspección de los datos**

Para entender mejor los datos, se utilizaron las siguientes **herramientas de visualización**:

- **Muestra de imágenes:**  
  Se reconstruyen algunas imágenes a partir de los valores de los píxeles para visualizar los dígitos.

- **Histograma de clases:**  
  Permite observar el **balance de clases** en el conjunto.

- **Matriz de correlación** (cuando aplica):  
  Útil en casos de datos tabulares, para ver la **relación entre variables**.

- **Gráficos de dispersión PCA o t-SNE:**  
  Se usaron para **reducir la dimensionalidad** y visualizar la distribución de las clases.

Estas visualizaciones ayudan a detectar problemas como **clases desbalanceadas**, **imágenes corruptas** o **falta de variedad** en los datos.

## **3. Preprocesamiento y preparación de los datos**

Las siguientes estrategias fueron aplicadas para preparar los datos:

- **Normalización:**  
  Se escalan los valores de píxeles entre 0 y 1 dividiendo por 255.

- **División en entrenamiento y prueba:**  
  Se dividió el conjunto en **80% para entrenamiento** y **20% para prueba**.

- **Codificación (cuando aplica):**  
  No necesaria aquí porque las etiquetas ya están codificadas como enteros.

- **Balanceo de clases:**  
  Se verificó que las clases estuvieran balanceadas. Si no lo estuvieran, se podría aplicar **sobremuestreo (SMOTE)** o **submuestreo**.

- **Reducción de dimensiones:**  
  Mediante **PCA** para acelerar algunos algoritmos, si se considera conveniente.

## **4. Entrenamiento de modelos y ajuste de hiperparámetros**

Se entrenaron los siguientes modelos utilizando **GridSearchCV** con validación cruzada:

- **Regresión logística multiclase (One-vs-Rest):**  
  Ajustando los parámetros **C** y **solver**.

- **Árboles de decisión:**  
  Ajustando **max_depth**, **min_samples_split** y **criterion**.

- **Random Forest:**  
  Optimizando **n_estimators**, **max_features** y **max_depth**.

- **Redes neuronales (MLPClassifier):**  
  Búsqueda sobre **hidden_layer_sizes**, **activation**, **alpha** y **learning_rate_init**.

Cada modelo fue evaluado usando **validación cruzada (cv=5)** para evitar sobreajuste y seleccionar los mejores hiperparámetros.